In [4]:
%run 02a_Preprocessing_and_feature_engineering.ipynb

In [5]:
serving = pd.read_csv('../datasets/test.csv')


## Fit the whole model


In [6]:
X_1_si = si.fit_transform(X_1)

In [7]:
serving.shape

(878, 80)

In [8]:
ols.fit(X_1_si, y)

LinearRegression()

## Create X Serving

In [9]:
replace_NaN_with_NA(serving, ['Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Fence'])

In [10]:
binarize_to_most_common_str_val(serving, [('Street', 'Paved Street'),
 ('Utilities', 'Has All Utilities'),
 ('Land Slope', 'Gentle Slope'),
 ('Central Air', None),
 ('Paved Drive', None)])

In [11]:
convert_num_to_cat(serving, ['PID', 'MS SubClass', 'Overall Qual', 'Overall Cond', 'Mo Sold'])

In [12]:
convert_years_to_ages(serving, 2010, [
    ('Year Built', 'Years since built'),
    ('Year Remod/Add', 'Years since remodel'),
    ('Yr Sold', 'Years since last sold')
])

In [13]:
# features_to_use_list.remove('Overall Qual_10')


In [14]:
serving.drop('PID', axis = 1 , inplace = True)

In [15]:
serving_dummy = pd.get_dummies(serving, columns = non_numerics.columns, drop_first = True)

Why is the line of code below here? This feature was present in both test and train but there were none in the "unseen" data! This caused a mismatch in columns and so I needed to add it in order for everything to line up

In [16]:
serving_dummy['Overall Qual_10'] = 0

In [17]:
X_serving = serving_dummy.loc[: , features_to_use_list]

## Transform X Serving

In [18]:
X_serving_si = si.transform(X_serving)

## Generate Predictions

In [19]:
salesprices = ols.predict(X_serving_si)

In [20]:
pd.DataFrame(salesprices).describe()

,0
count,878.000000
mean,179015.150779
std,70047.417797
min,47610.595706
25%,128589.555461
50%,165584.037181
75%,210898.079005
max,438725.966557


In [21]:
salesprices.shape

(878,)

In [22]:
serving['SalePrice'] = salesprices

In [23]:
submission1 = serving.loc[:, ['Id','SalePrice']]

In [24]:
submission1.set_index('Id', inplace = True)

In [25]:
submission1.to_csv('../datasets/submission1_ols.csv')